# 线性回归实践

在这个实践中，我们将学习如何使用PyTorch实现线性回归模型。我们将以预测混凝土强度为例，这是土木工程中的一个常见问题。

## 1. 导入必要的库

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## 2. 准备数据

在实际项目中，数据通常来自实验或实地测量。这里，我们将创建一些模拟数据来演示这个过程。

假设我们有一组数据，表示水泥用量(kg/m³)和对应的28天抗压强度(MPa)。

In [ ]:
# 生成模拟数据
np.random.seed(42)
cement = np.random.uniform(200, 400, 100)  # 水泥用量
strength = 0.1 * cement + 10 + np.random.normal(0, 5, 100)  # 抗压强度

# 转换为PyTorch张量
X = torch.FloatTensor(cement.reshape(-1, 1))
y = torch.FloatTensor(strength.reshape(-1, 1))

# 划分训练集和测试集
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 可视化数据
plt.scatter(X, y)
plt.xlabel('cement consumption(kg/m³)')
plt.ylabel('compression strength at 28th day (MPa)')
plt.title('Concrete Compressive Strength wrt. Cement Consumption')
plt.show()

## 3. 定义模型

我们将使用PyTorch的`nn.Module`来定义我们的线性回归模型。

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.linear(x)

# 实例化模型
model = LinearRegression(1, 1)
print(model)

## 4. 定义损失函数和优化器

对于回归问题，我们通常使用均方误差(Mean Squared Error, MSE)作为损失函数。
优化器用于更新模型参数，这里我们使用随机梯度下降(SGD)。

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## 5. 训练模型

现在我们开始训练模型。在每个epoch中，我们会:
1. 进行前向传播得到预测值
2. 计算损失
3. 反向传播计算梯度
4. 更新模型参数

In [ ]:
num_epochs = 50
losses = []

for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 绘制损失曲线
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

## 6. 评估模型

训练完成后，我们使用测试集来评估模型的性能。

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# 可视化结果
plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.scatter(X_test, y_pred, color='red', label='Predicted')
plt.xlabel('cement consumption (kg/m³)')
plt.ylabel("compression strength at 28th day (MPa)")
plt.title('real vs predicted compressive strength')
plt.legend()
plt.show()

## 7. 使用模型进行预测

现在我们可以使用训练好的模型来预测新的数据。

In [ ]:
# 假设我们要预测水泥用量为350 kg/m³时的混凝土强度
new_cement = torch.FloatTensor([[350]])
predicted_strength = model(new_cement).item()
print(f'当水泥用量为350 kg/m³时,预测的28天抗压强度为: {predicted_strength:.2f} MPa')

## 8. 总结

在这个实践中，我们学习了如何使用PyTorch实现一个简单的线性回归模型来预测混凝土强度。主要步骤包括:

1. 准备和可视化数据
2. 定义模型结构
3. 选择损失函数和优化器
4. 训练模型
5. 评估模型性能
6. 使用模型进行预测

这个例子展示了如何将深度学习应用于土木工程中的实际问题。通过这种方法，我们可以建立水泥用量和混凝土强度之间的关系模型，这对于混凝土配比设计和质量控制都有重要意义。

在实际应用中，我们可能需要考虑更多的因素(如水灰比、骨料类型等),这时可以扩展到多元线性回归或更复杂的非线性模型。